In [ ]:

import numpy as np
import pandas as pd
import nltk
from random import sample
import torch
torch.cuda.empty_cache()
import os
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from random import sample
from rouge import Rouge

In [ ]:
path = '/content/drive/MyDrive/CNN/train.csv'
trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)

MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()
trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])


<ipython-input-2-63dc1f168511>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)


In [ ]:
for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('===========================================================================================================\n\n')

Original Text : 
By . Daily Mail Reporter . A giant coffee shop billboard featuring a naked woman is brewing controversy in Washington, with locals demanding the racy sign be pulled down. A picture of a blonde woman bearing her rear in a tiny g-string adorns the 10-foot-tall mobile billboard, parked on the side of various busy highways to promote two lingerie coffee bars. While the unidentified business owner insists she has a right to advertise her coffee stands as she chooses, some locals are disgusted at the graphic nudity. 'It's completely unacceptable and I am offended,' mother-of-four Susan Evans told KXLY.com, noting she once ate with her family across the street from the sign. 'This must be taken down.' Scroll down for video . Controversy brewing: Some locals are outraged at this billboard for 2nd Base Lingerie Espresso which features a nude woman with wings on a Washington state highway . It moves: The traveling billboard is shuttled between two Washington coffee stands - 2nd 

In [ ]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
MAX_EPOCHS = 5

torch.cuda.memory_summary(device=None, abbreviated=False)
torch.utils.checkpoint

model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
            eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE],
            source_max_token_len=MAX_LEN,
            target_max_token_len=SUMMARY_LEN,
            batch_size=4, max_epochs=MAX_EPOCHS, use_gpu=True)

model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

model.load_model("t5","./"+model_path, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

outputs/simplet5-epoch-4-train-loss-1.0175-val-loss-1.5973


In [ ]:
no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [ ]:
for index in sample(list(np.arange(len(trainning_article_ls))),5):
    print('Original Text : ')
    print(trainning_article_ls[index])

    Original_article = trainning_article_ls[index]

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    Original_Summary = trainning_highlight_ls[index]

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))

    Predicted_Summary = trainning_article_ls[index]

    rouge = Rouge()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([Predicted_Summary], Original_Summary)

    print("-----------------")

    print(rouge.get_scores(Predicted_Summary, Original_Summary))

    print("-----------------")

    print(BLEUscore)

    print("-----------------")

    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Original Text : 
A bureau de change boss who recruited his wife and son to help clean up nearly £170 million-worth of criminal cash has been jailed for nine years. Ussama El-Kurd, 64, known as 'Sammy the Kurd' in underworld circles, was on bail pending an appeal for a previous money laundering conviction when he set up the business in 2006. He enlisted his wife Randa, 51, and son Husain, 22, to help run the Kensington Money Exchange in upmarket Notting Hill, west London. Ussama El-Kurd, known as 'Sammy the Kurd', was on bail for a previous money laundering conviction when he set up the business in 2006 . Their efforts allowed the proceeds of drug dealing, violent robberies and other major crimes to be spent freely by offenders. The Palestinian-born family ran their 'bespoke money laundering service' in Kensington Church Street. No records of the dodgy transactions were kept, but authorities say the business must have helped to wash at least £169m until it was shut down in 2010. They es